In [ ]:
from elasticsearch import helpers, Elasticsearch
import elasticsearch

In [ ]:
es = Elasticsearch('http://localhost:9200', max_retries=10, retry_on_timeout=True)

In [ ]:
df = spark.read.csv('DNPBA2017.csv', header=True)
df.limit(2).toPandas()

### indexing

In [ ]:
es_index_name = "matb10-sinasc"
df.write.format("org.elasticsearch.spark.sql").option("es.resource", es_index_name).mode('overwrite').save()

### buscando inexata termo

In [ ]:
content = { "bool": 
           { "should": [ {"term": {"CODMUNNASC":"291170"}} ] 
           } 
          }

In [ ]:
res = es.search(index=es_index_name, query=content, size=2)

In [ ]:
res['hits']['hits']

<hr />

### buscando exata termo

In [ ]:
content = { "bool": 
           { "must": [ 
               {"match": {"CODESTAB": "2786095"}},
               {"term": {"CODMUNNASC":"170700"}}] 
           } 
          }

In [ ]:
res = es.search(index=es_index_name, query=content, size=2)

In [ ]:
res['hits']['hits']

### buscando um intervalo

In [ ]:
df.select("IDADEMAE").groupBy("IDADEMAE").count().orderBy("IDADEMAE").show(30)

In [ ]:
content = {"range": {
            "IDADEMAE": {
                "gte": 20,
                "lt": 30
            }
        }
    }

In [ ]:
res = es.search(index=es_index_name, query=content, size=2)

In [ ]:
res['hits']['hits']

### busca lógica

In [ ]:
content = {
    "bool": {
            "must": [
                {
                    "term": {
                        "IDADEMAE": 21
                    }
                },
                {
                    "term": {
                        "ESCMAEAGR1": "06"
                    }
                }
            ]
        }
    }

In [ ]:
res = es.search(index=es_index_name, query=content, size=2)

In [ ]:
res

### searching methods

In [ ]:
def buscaExata(numerodn, codestab, idademae, escmaeagr1, startId=0): 
    
    global es
    
    content = {
        'bool': {
                'must': [
                    {'match_phrase': {'NUMERODN': '"' + numerodn + '"'}},
                    {'match_phrase': {'CODESTAB': '"' + codestab + '"'}}, 
                    {'match': {'IDADEMAE': idademae}},
                    {'match': {'ESCMAEAGR1': escmaeagr1}}
                ]
            }
        }
    force = True
    while force:
        try:
            res = es.search(index=es_index_name, query=content, size=2)
            force = False
        except:
            pass
    return res['hits']['hits']

In [ ]:
def buscaAproximada(numerodn, codestab, idademae, escmaeagr1, startId=0):
    
    global es
    
    content = {
        'bool': {
                'should': [
                    {'match': {'NUMERODN': {'query': numerodn, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'CODESTAB': {'query': codestab, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'IDADEMAE': {'query': idademae, 'fuzziness':'AUTO', 'operator':'or', 'boost':'0.5'}}},
                    {'match': {'ESCMAEAGR1': {'query': escmaeagr1, 'fuzziness':'AUTO', 'operator':'or'}}}
                ]
            }
        }
    force = True
    while force:
        try:
            res = es.search(index=es_index_name, query=content, size=15)
            force = False
        except:
            pass
    return res['hits']['hits']

In [ ]:
buscaExata("72390242", "2786095", "25", "01")#[0]['_source']['NUMERODN']

In [ ]:
# um campo modificado (note que o retorno é vazio)
buscaExata("72390242", "2786095", "33", "01")

In [ ]:
buscaAproximada("72390242", "2786095", "25", "01")[4]['_source']['NUMERODN']

In [ ]:
# # um campo modificado (mais de um candidato é retornado)
buscaAproximada("72390242", "2786095", "33", "01")

In [ ]:
print(len(buscaAproximada("72390242", "2786095", "33", "01")))